# Practical session 2: Implementing reactive behaviors

## Preliminary notes

**Reminders :** 
- Save your work as indicated at the beginning of the `session_1.ipynb` notebook.
- Each time you encounter a cell containing code in this notebook (as in the cell starting with `from vivarium.controllers...` below), you have to execute it by clicking on the cell and pressing `Shift+Enter` (unless it is explicitly specified not to do it). This will import the necessary modules and functions to use the simulator.

<!-- TODO: load scene !-->

In [1]:
from vivarium.controllers.notebook_controller import NotebookController
from vivarium.utils.handle_server_interface import start_server_and_interface, stop_server_and_interface

Then, launch the simulator for this session as well as the interface : 

In [2]:
start_server_and_interface(scene_name="session_2")

/Users/clement/Documents/work_nocloud/dev/vivarium/vivarium/utils

STARTING SERVER
[2024-12-06 12:55:53,591][__main__][INFO] - Scene running: session_2
[2024-12-06 12:55:55,687][vivarium.simulator.simulator][INFO] - Simulator initialized

STARTING INTERFACE


Wait until the interface link shows up (http://localhost:5006/run_interface) and click on it, and make sure the scene is present on your browser. 

Create a controller that will be used to control the simulation with Python code from this jupyter notebook:

In [3]:
controller = NotebookController()

/Users/clement/Documents/work_nocloud/dev/vivarium/env_vivarium/lib/python3.10/site-packages/jax/_src/ops/scatter.py:96: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=float32 to dtype=int32 with jax_numpy_dtype_promotion='standard'. In future JAX releases this will result in an error.
  warnings.warn(


Then start the controller:

In [4]:
controller.run()

2024-12-06 12:55:59,324 Starting Bokeh server version 3.3.4 (running on Tornado 6.4.1)
2024-12-06 12:55:59,325 User authentication hooks NOT provided (default user enabled)
2024-12-06 12:55:59,326 Bokeh app running at: http://localhost:5006/run_interface
2024-12-06 12:55:59,326 Starting Bokeh server with process id: 44911


We will use a single agent in this session, let's create an alias for it as we did in Session 1:

In [5]:
agent = controller.agents[0]

If at some point things are not going as expected (e.g. the agent doesn't move the way it should in the simulator), follow the steps explaine in Session 1 or in the [troubleshooting tutorial](../tutorials/troubleshooting.ipynb).

## Behaviors

In the last practical session we saw how to set the agent left and right motor speeds as well as how to read the values returned by the left and right proximeters. We programmed a first simple behavior where the agent slows down when approaching an obstacle. 

Here is a possible solution for this behavior, if will run for 20 seconds if you execute the next cell:

In [ ]:
# Repeat 200 times the indented code:
for i in range(200):
    # print the iteration number every 20 iterations
    if i % 20 == 0:
        print(f"Iteration {i}")
        
    # Read the proximeter values and store them in the "left" and "right" variables
    left, right = agent.sensors()
    
    # Compute the maximum of the values returned by the left and right proximeters.
    max_prox_value = max(left, right)
    
    # Compute the activation that will be applied to both motors. 
    # The closer the obstacle (i.e. the higher the max value of the proximeters), the lower the motor activation should be.
    # Note that motor activation is bounded between 0 and 1
    motor_activation = 1.0 - max_prox_value
    
    # Set the activation of both motors to the value we just have computed
    agent.left_motor = motor_activation
    agent.right_motor = motor_activation
    
    # Waits for 100 milliseconds before starting the next iteration (to avoid overloading you computer)
    controller.wait(0.1)

# Stop the robot
agent.stop_motors()

Iteration 0
Iteration 20
Iteration 40
Iteration 60
Iteration 80
Iteration 100
Iteration 120
Iteration 140
Iteration 160
Iteration 180


## Practical definition of a behavior

The example behavior defined above illustrates the general structure of a behavior. 

**Definition:** a behavior consists of a loop repeated at a certain frequency where (1) the values of relevant sensors are read, (2) some computation is performed using these values and (3) commands are sent to the agent motors according to the result of this computation.

In the example behavior above, step (1) corresponds to the reading of the left and right proximeters activations. Step (2) corresponds to the computation of `motor_activation` according to the maximum of the proximeter activations. Finally, Step (3) corresponds to setting the speed of both motors to the value of `motor_activation`.

Note that the code above will take a while to be executed (approximately `200 * 0.1 = 20` seconds, since the loop is repeated 200 times). During this time, you can't execute anything else in this notebook. To stop the execution before it terminates by itself you have manually stop the cell execution (by pressing the "stop-like" button, located either in the top menu bar of this document or next to the executing cell).  

This approach has three major drawbacks:
- Only one behavior can run at a time.
- The behavior has a fixed duration (at one point it will stop)
- We can't stop a behavior programmatically (instead we have to press the "stop-like" button).

To overcome these problems, we provide a more flexible method for defining and executing behaviors. Let's rewrite the behavior above using that method. First make sure the previous code is not still being executed by pressing the "stop-like" button. Now, defining a behavior boils down to defining a function which includes the core of the behavioral loop:

In [6]:
# The code in this cell defines a function called slow_down (first line),
# which takes as argument the agent (first line, in parenthesis),
# and returns the left and right wheel activation to be applied to the motors (last line)

def slow_down(agent):
    # Step (1): read the sensor values
    left, right = agent.sensors()
    
    # Step (2): do some computation
    max_prox_value = max(left, right)
    motor_activation = 1.0 - max_prox_value
    
    # Step (3): return the motor activations for left and right motors
    return motor_activation, motor_activation

The cell above defines a function called `slow_down`. In computer programming, a function is a sequence of instructions that perform a specific task depending on some parameters (called the arguments of the function) and that returns a result. In this sense it is very similar to the mathematical definition of a function, as for example when we write `y = f(x)`, where `f` is the name of the function, `x` is its argument, and `y` is the result. For example, we can define a function `square` that computes the square of its argument as follows:


In [13]:
def square(x):
    return x * x

print(square(3))

print(square(5))

9
25



As seen above, the definition of a function in Python starts with the keyword `def`, followed by an arbitrary name we choose for the function (here we called it `slow_down` to reflect the purpose of the behavior defined in it). Then come the arguments of the function in parenthesis (in our case it will be the variable representing the agent, called `agent`) and finally the symbol `:`. Below the first line, you find the instructions that this function will execute when it will be called. Those instructions need to be intended right with respect to the first line. In this example, the instructions are the exact same as in the core of the previous `for` loop, except that:
- we omit the last line `controller.wait(0.1)` (the frequency at which the behavior will be executed will be set in more rigorous way below),
- we don't directly set the motor activations using `agent.left_motor` and `agent.right_motor`. Instead, we *return* the values of the motor activations in the last line and they will be automatically sent to the agent motors when the behavior will be executed. In the last line, the values after the `return` keyword have to be the left and right wheel activation (in this order). Both activations have to be between 0 and 1. (In the `slow_down` behavior above, both activations are the same since we don't want the agent to turn).

Note that a function definition, as the one above, does not execute the instructions contained in it, it only defines them so that they can be executed later when the function will be *called*. In our case, we will not explicitly call the function, instead it will be done behind the scene when we will start the behavior on the agent (see below).

Once the behavior is defined as a function, we can attach it to the agent by executing:


In [17]:
agent.attach_behavior(slow_down)

The line above means: attach the behavior defined in the function `slow_down` to the `agent` and execute it. You should now see the agent executing the exact same behavior as before (if the agent is already close to an obstacle, move it to a more open space to observe it slowing down).

Executing the above line will basically do the same thing as executing the `for` loop at the start of this document. Here, the function `slow_down` will be executed indefinitely. 

Using this method has the following advantages over the previous method using the `for` loop:
- It is more compact to write and it will allow to better structure your code when you will have to deal with multiple behaviors and multiple agents.
- It is not blocking as the previous method was. This means that you can still use this notebook while the behavior is running on the agent. For example, let's read the proximeter activations while the agent is still executing the `slow_down` behavior:

In [11]:
agent.sensors()

[0.4055134057998657, 0.0]

Each time you execute the cell above, you should see the proximeter activation changing because the agent is moving. However, you should avoid setting motor values while a behavior is running since this could conflict with the behavior also setting those values. When a behavior is attached, it runs indefinitely until you explicitly detach it. To do so, you have to execute:

In [18]:
agent.detach_behavior("slow_down")

Note that the agent will continue moving using the last motor speeds that were set by the behavior. You can set both motor speeds to 0 by executing:

In [19]:
agent.stop_motors()

You can also stop the behavior and stop the motors of the agent in the same line. To demonstrate this, let's first attach the behavior again by executing:

In [21]:
agent.attach_behavior(slow_down)

Now we detach the behavior, this time setting the `stop_motors` argument to `True` in the `detach_behavior` function:

In [22]:
agent.detach_behavior("slow_down", stop_motors=True)

The stop_motors argument will automatically execute agent.stop_motors() before detaching the behavior. If you don't want the motors to stop when detaching the behavior, you can set stop_motors to False.

## More detail on behavior management (optional)
<!-- Do we make this optional? -->

By default, `attach_behavior` also executes the behavior on the agent it has been attached to. If you don't want to start the behavior when attaching it for some reason, you can set the optional argument `start` to `False` as follows:

In [23]:
agent.attach_behavior(slow_down, start=False)

Now the agent is "aware" of the `slow_down` behavior, but does not execute it. You can check it in the interface: the agent does not move. To execute the behavior you need to explicitely start it:

In [24]:
agent.start_behavior("slow_down")

This can be useful when you need a better control on when to start or stop behaviors, for example if you want to attach multiple behaviors to the agent and start them all at the same time (will see how to do this in the next session). You can also stop a behavior while keeping it attached with:

In [25]:
agent.stop_behavior("slow_down", stop_motors=True)

At anytime, you can check what behaviors are attached or started with:

In [26]:
agent.print_behaviors()

Attached behaviors: ['slow_down'], Started behaviors: No started behaviors


Here it indicated that the `slow_down`is currently attached, but not started.

**Q1:** To make sure you correctly understand how to attach, start, stop and detach a behavior, complete the following code:

In [7]:
# First we make sure that no behavior is attach to the agent:
agent.detach_all_behaviors()

# When checking, it should print "No behavior attached":
print("\nstep 1:")
agent.print_behaviors() # This will print "No behavior attached"

# Write just below this line the code that attaches and start the slow_down behavior
agent.attach_behavior(slow_down)
print("\nstep 2:")
agent.print_behaviors() # This will print "Attached behaviors: ['slow_down'], Started behaviors: ['slow_down']"

# Write just below this line the code that detaches the slow_down behavior
agent.detach_behavior("slow_down")
print("\nstep 3:")
agent.print_behaviors() # This will print "No behavior attached"

# Write just below this line the code that attaches the slow_down behavior but does not start it
agent.attach_behavior(slow_down, start=False)
print("\nstep 4:")
agent.print_behaviors() # This will print "Attached behaviors: ['slow_down'], No started behavior"

# Write just below this line the code that starts the slow_down behavior
agent.start_behavior("slow_down")
print("\nstep 5:")
agent.print_behaviors() # This will print "Attached behaviors: ['slow_down'], Started behaviors: ['slow_down']"

# Write just below this line the code that stops the slow_down behavior
agent.stop_behavior("slow_down")
print("\nstep 6:")
agent.print_behaviors() # This will print "Attached behaviors: ['slow_down'], Started behaviors: No active behaviors"

# Write just below this line the code that detaches the slow_down behavior
agent.detach_behavior("slow_down")
print("\nstep 7:")
agent.print_behaviors() # This will print "No behavior attached"



step 1:
No behaviors attached

step 2:
Attached behaviors: ['slow_down'], Started behaviors: ['slow_down']

step 3:
No behaviors attached

step 4:
Attached behaviors: ['slow_down'], No behavior started

step 5:
Attached behaviors: ['slow_down'], Started behaviors: ['slow_down']

step 6:
Attached behaviors: ['slow_down'], No behavior started

step 7:
No behaviors attached


By default, the behaviors of the agents will be executed at every time step of the simulation. You can choose a different interval of behavior execution by setting the `interval` argument when attaching the behavior to the agent. For example, to execute the `slow_down` behavior every 10 steps of the simulation, you can execute:

We recommend to not change the default interval unless the simulator becomes slow. This should not be the case for now, but it might be when we learn to program more complex simulation. 

You can check how many step of the simulation is currently executed per second with:

In [29]:
controller.print_fps()

measuring the FPS (number of steps per second) in the controller during 2 seconds ...


FPS: 69.50


## Summary: managing behaviors the simple way

Let's summarize the method we have just describe to define, attach and detach a behavior:

In [ ]:
# First, detach all the behaviors that might still be attached to the agent
# (it is a good practice to do it each time you want to define a new behavior, or modify an existing one):
agent.detach_all_behaviors(stop_motors=True)

In [31]:
# Define a behavior where the agent progressively slows down when it approaches an obstacle:
def slow_down(agent):
    # Step (1): read the sensor values
    left, right = agent.sensors()
    
    # Step (2): do some computation
    sum_of_proxs = left + right
    motor_activation = (2 - sum_of_proxs) / 2
    
    # Step (3): return the motor activations
    return motor_activation, motor_activation

In [ ]:
# Attach and start this behavior to the agent, and specify the step interval at which it will be executed
agent.attach_behavior(slow_down)

When executing the code above, you should see the behavior being executed on the agent in the simulator. Then, to detach the behavior:

In [ ]:
agent.detach_behavior(slow_down, stop_motors=True)

An alternative way is to stop and detach all the behaviors running on the agent. This avoids having to specify the name of the behavior (`slown_down` in the cell above) and also stops systematically the behavior before detaching it:

+ stop motors (should me make it a default arg with this function ? )

In [35]:
agent.detach_all_behaviors(stop_motors=True)

## Implementing the Braitenberg vehicles

Let's now practice a bit. Remember the Braitenberg Vehicle examples we have seen in [this slide](https://docs.google.com/presentation/d/1s6ibk_ACiJb9CERJ_8L_b4KFu9d04ZG_htUbb_YSYT4/edit#slide=id.g31e1b425a3_0_0). Those vehicles are very similar to the agents in the simulator. 
- A Braintenber Vehicle is equipped with two sensors that are activated according to the proximity of a source. With the agent, each proximeter sensor returns a value between 0 and 1 that is inversely proportional to the distance from the closest obstacle it perceives (the closer the obstacle, the highest to proximeter activation). Sensor values are accessed with `agent.sensors()`
- A Braintenber Vehicle is equipped with two wheels. An agent in the simulator is also equipped with two whells, whose rotating speeds are controlled through the activation of each motor independently with a value between 0 and 1 (where 1 means maximum speed). E.g. setting the left wheel at full speed is achieved with `agent.left_motor = 1`, while stopping the right wheel is achieved with `agent.right_motor = 0`.
- A behavior associates sensor activations to motor activations. In the Braitenberg Vehicles, this is achieved through connections that are either excitatory (the activity of the sensor increases the activity of the motor it is connected to) or inhibitory (the activity of the sensor decreases the activity of the motor it is connected to). In the agent, we have seen above that we can define a behavior as a function that (1) read the sensor activities (2) perform some computation and (3) use the result of that computation to set the motor speed. 

Therefore, we can implement in the agent the various types of vehicle behaviors shown in the slide, where defining excitatory and inhibitory connections will be done through Step (2) above (*perform some computation*). We have actually already done it with the `slow_down` behavior we have defined above.  

**Q2:** Define verbally the `slow_down` behavior in term of inhibitory and excitatory connections (do it by double clicking on the next cell). Your answer must look like this (where `TO_FILL` is either the word "excitatory" or "inhibitory"):
- The activity of the left sensor is connected to the left motor through a TO_FILL connection.
- The activity of the left sensor is connected to the right motor through a TO_FILL connection.
- The activity of the right sensor is connected to the left motor through a TO_FILL connection.
- The activity of the right sensor is connected to the right motor through a TO_FILL connection.

*Double click on this cell and replace this text by your answer*

Let's see how to define the `fear` behavior illustrated in [the slide](https://docs.google.com/presentation/d/1s6ibk_ACiJb9CERJ_8L_b4KFu9d04ZG_htUbb_YSYT4/edit#slide=id.g31e1b425a3_0_0) using the method we have seen: 

In [8]:
def fear(agent):
    left, right = agent.sensors()
    return left, right

That's pretty easy, isn't it? As illustrated in the slide, the `fear` behavior simply consists in the left sensor exciting the left motor, and the right sensor exciting the right motor. Therefore, the simplest way of programming this behavior is to directly map the left and right sensor activations to the left and right motor speed, respectively. This is what is done in the function definition just above. Since both sensor and motor values are bounded between 0 and 1, there is nothing else to take care of.

Let's now analyze the properties of this `fear` behavior in more detail. Attach and start the `fear` behavior by executing the cell below, and observe how the agent behaves.

In [9]:
agent.detach_all_behaviors()  # Just in case a behavior is still attached

agent.attach_behavior(fear)
agent.start_all_behaviors()

2024-12-06 14:35:20,690 WebSocket connection opened
2024-12-06 14:35:20,704 ServerConnection created


Note that this behavior will make the robot move only if at least one if its sensor detects an object (ojects are represented as squares in the interface). If no object is with the agent's field of view, try to move either the agent or some objects (see the [web interface tutorial](../tutorials/web_interface_tutorial.md) for how to move entities in the interface).

**Q3:** Use the cell below to answer the following questions.
<!-- TODO: change that because there is no walls/corners !-->
1. What happens when the activity of both sensors is null? (i.e. no object is detected.) Why?
2. How does the agent react when it detects an object? Why?
3. Imagine a small animal equipped with such a behavior in the wild. What would be its evolutionary advantages and drawbacks? (could it escape from a predator? could it collect food? Could it hide itself?)

*Double click on this cell and replace this text by your answer*

**Q4:** Program the `aggression` behavior illustrated in [the slide](https://docs.google.com/presentation/d/1s6ibk_ACiJb9CERJ_8L_b4KFu9d04ZG_htUbb_YSYT4/edit#slide=id.g31e1b425a3_0_0), which consists of crossed excitatory connections. 

In [10]:
def aggression(agent):
    left, right = agent.sensors()
    return right, left

Before executing the behavior you have defined in the cell just above, first detach the previous one and immobilize the agent:

In [11]:
agent.detach_all_behaviors(stop_motors=True)

Then attach the `aggression` behavior:

In [ ]:
agent.attach_behavior(aggression)


**Q5:** Use the cell below to answer the following questions.

3. How does the agent reacts when it approaches an object?  Why?
2. How does the agent react when close to a moveable object (the orange squares in the scene) Why?
4. Imagine an animal equipped with such a behavior in the wild. What would be its evolutionary advantages and drawbacks? (could it escape from a predator? could it catch preys? Could it hide itself? Could it move things?)

*Double click on this cell and replace this text by your answer*

That's it for this practical session. You can now close the session:

In [13]:
controller.stop()
stop_server_and_interface(safe_mode=False)

 Found the process scripts/run_interface.py running with this PID: 44911
 Found the process scripts/run_server.py running with this PID: 44900
Killed process with PID: 44911
Killed process with PID: 44900

Server and Interface processes have been stopped



Simulator is already stopped

Stopping server and interface processes

Received signal 15, shutting down


False

Now that you finished session 2, you can now jump to the notebook of [session 3](session_3.ipynb).